### HashTag Creator - Basic Solution
By Ivelize

This solution is focused on show how an inverted index problem could be solve without API.

#### Problem definition: given a set of txt files, find the "most common" occurring words, and the sentences where they are used.

Note: The algorithm's criteria to sort the words is: (i) the words which occur in the highest quantity in terms of files number, and (ii) the words which occur in the highest quantity in terms of absoluty number.

#### Assumption: 
 - the files must be in the same notebook's directory.

In [37]:
from collections import Counter
from collections import OrderedDict
import pandas as pd
import re
import glob

In [38]:
stopwords = {"a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "don't", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "it's", "its", "itself", "just", "keep", "last", "latter", "latterly", "let", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "that's", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "we're", "we've", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "-"}

In [39]:
# Reading the txt files
list_setences_per_file = list()
dict_words_frequency_per_file = {}
for filename in glob.glob('*.txt'):
    with open(filename) as f:
        words_frequency = Counter(f.read().split())
        f.seek(0)
        sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', f.read())
    # remove whitespace characters like `\n` at the end of each line
    sentences = [x.strip() for x in sentences]
    list_setences_per_file.extend(sentences)
    dict_words_frequency_per_file[filename.split('.')[0]] = words_frequency
    f.close()

In [40]:
dict_words_cleaned = {}
for fileid, words_freq in dict_words_frequency_per_file.iteritems(): 
    for k,v in words_freq.iteritems():
        word = k.lower()      
        if word not in stopwords:
            info={}
            if word in dict_words_cleaned.keys():
                value = dict_words_cleaned.pop(word)
                info['files'] = value['files']
                if fileid not in value['files']:
                    info['files'].append(fileid)
                info['sentence'] = []
                info['file_freq'] = value['file_freq'] + 1
                info['word_freq'] = value['word_freq'] + v
                dict_words_cleaned[word] = info
            else:
                info['files'] = [fileid]
                info['sentence'] = []
                info['file_freq'] = 1
                info['word_freq'] = v 
                dict_words_cleaned[word]= info

In [41]:
for k,v in dict_words_cleaned.items():
    for sentence in list_setences_per_file:
        if k in sentence:
            dict_words_cleaned[k]['sentence'].append(sentence)

In [42]:
dict_words_cleaned = OrderedDict(sorted(dict_words_cleaned.items(), key=lambda x: (x[1]['file_freq'], x[1]['word_freq']), reverse=True))

In [43]:
df = pd.DataFrame(dict_words_cleaned, index=['files', 'sentence'])

In [44]:
df

,time,like,right,tell,good,"now,",thank,people,american,government,...,inspiration,bulwark,resolution,baker,"slumber,",craft,diploma.,news.,dismissal,"""hey,"
files,"[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc5]","[doc2, doc3, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]","[doc2, doc3, doc1, doc6, doc4, doc5]",...,[doc2],[doc5],[doc5],[doc5],[doc1],[doc5],[doc2],[doc2],[doc4],[doc6]
sentence,"[I became a civil rights lawyer, and taught co...","[We welcomed immigrants to our shores, we open...","[I became a civil rights lawyer, and taught co...","[But let me tell you how I came to be here., I...",[And if you will join me in this improbable qu...,"[As most of you know, I am not a native of thi...",[Let me begin by saying thanks to all you who'...,[In the face of a politics that's shut you out...,[],[I saw that the problems people faced weren't ...,...,[Let me express my thanks to the historic slat...,[The notion that Iraq would quickly and easily...,"[At that point, seventy-five U.S. Senators, Re...",[],[And if you will join me in this improbable qu...,[I am committed to working with this White Hou...,[We measure progress by how many people can fi...,[This country is more generous than one where ...,"[This coalition succeeded by promising change,...",[]
